In [ ]:
import asyncio
import logging
from typing import Annotated
from semantic_kernel.agents import (
    ChatCompletionAgent,
    ChatHistoryAgentThread,
    AgentGroupChat
)
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel import Kernel
from semantic_kernel.functions import kernel_function, KernelArguments
from semantic_kernel.connectors.ai import FunctionChoiceBehavior
from semantic_kernel.agents.strategies import TerminationStrategy, KernelFunctionTerminationStrategy, KernelFunctionSelectionStrategy
from semantic_kernel.functions import KernelFunctionFromPrompt
from semantic_kernel.contents import ChatHistoryTruncationReducer

from dotenv import load_dotenv
load_dotenv(override=True)
#logging.basicConfig(level=logging.INFO)

In [ ]:
class verifyIdentity:
    """本人確認をするプラグインです"""

    @kernel_function(description="銀行カード番号をキーとして顧客マスターの情報を取得します")
    def get_clientMaster_by_bankCardNumber(
        self, bankCardNumber: Annotated[str, "銀行カード番号"]
    ) -> Annotated[str, "顧客マスターから、氏名、住所、電話番号を返します"]:
        return  """
        マイクロ太郎
        港区港南 2-16-3
        03-4332-5300
        """

class addressUpdate:
    """住所変更をするプラグインです"""

    @kernel_function(description="銀行カード番号をキーとして顧客マスターの住所変更をします")
    def update_clientMaster_by_bankCardNumber(
        self, bankCardNumber: Annotated[str, "銀行カード番号"], newAddress: Annotated[str, "新しい住所"]
    ) -> Annotated[str, "顧客マスターから、氏名、住所、電話番号を返します"]:
        return  """
        マイクロ太郎
        港区港南 9-86-7
        03-4332-5300
        """

In [ ]:
# 各エージェントレベルのインストラクションで終了条件を指定する必要はありません。
RECEPTIONIST_NAME = "Receptionist"
RECEPTIONIST_INSTRUCTIONS = """
あなたは、銀行の受付係で、顧客からのリクエストを受付け、完了するまでのプロセスを管理します。
あなたの目標は、顧客からリクエストをヒアリングし、タスクを適切な担当者に振り分け、顧客のリクエストが解決したかどうかを確認することです。
現在銀行が行えるタスクは住所変更のみです。

あなたは、顧客からのリクエストを受け取ったら、まず本人確認を行うために、顧客に銀行カード番号を聞いてください。
銀行カード番号をキーとして、顧客マスターの情報を取得し、顧客の氏名、住所、電話番号を返し、正しいかどうかを確認してください。

"""


ADDRESSUPDATER_NAME = "AddressUpdater"
ADDRESSUPDATER_INSTRUCTIONS = """
あなたは顧客の住所変更をする担当者です。
あなたのタスクは、顧客から住所変更に必要な書類を提示してもらい、顧客マスターに登録されている顧客の住所を変更することです。
まず、顧客から住所変更に必要な書類を提示してもらってください。
タスクが完了したら、更新された顧客マスターの氏名、住所、電話番号を返してください。
住所変更に必要な書類は次のいずれかです。
- 運転免許証
- 健康保険証

"""

OTHERTASKOPERATOR_NAME = "OtherTaskOperator"
OTHERTASKOPERATOR_INSTRUCTIONS = """
あなたはその他のタスクを担当する担当者です。
あなたは現在何も担当していません。
リクエストが来たら、何もせずに「おやすみなさい」と返してください。
"""


In [ ]:
# このシナリオならカーネルは共通でも良い
def _create_kernel_with_chat_completion(service_id: str) -> Kernel:
    kernel = Kernel()
    kernel.add_service(AzureChatCompletion(service_id=service_id, deployment_name="gpt-4o"))
    return kernel

agent_receptionist = ChatCompletionAgent(
    kernel=_create_kernel_with_chat_completion(RECEPTIONIST_NAME),
    name=RECEPTIONIST_NAME,
    instructions=RECEPTIONIST_INSTRUCTIONS,
    plugins=[verifyIdentity()],
)

agent_addressupdater = ChatCompletionAgent(
    kernel=_create_kernel_with_chat_completion(ADDRESSUPDATER_NAME),
    name=ADDRESSUPDATER_NAME,
    instructions=ADDRESSUPDATER_INSTRUCTIONS,
    plugins=[addressUpdate()],
)

agent_othertaskoperator = ChatCompletionAgent(
    kernel=_create_kernel_with_chat_completion(OTHERTASKOPERATOR_NAME),
    name=OTHERTASKOPERATOR_NAME,
    instructions=OTHERTASKOPERATOR_INSTRUCTIONS,
)

### KERNEL Function Strategy

In [ ]:
# 完了条件のプロンプトでどういう結果を返すか指示します。
# この指示が result_parser で判定されます。
termination_function = KernelFunctionFromPrompt(
    function_name="termination",
    prompt="""
    顧客のリクエストが完了したか、もしくは顧客に質問するためにいったん会話を中断するかどうか判断します。
    顧客のリクエストが完了した場合は "<request_completed>" を返してください。
    顧客のリクエストが完了していない場合は "<return_to_user>" を返してください。
    それ以外の場合はエージェントの対応が引き続き必要です。

    History:
    {{$history}}
    """,
)

# 選択条件である程度のプロセスを書いた方が良さそうです
selection_function = KernelFunctionFromPrompt(
    function_name="selection",
    prompt=f"""
    会話の次の順番に進む参加者を、最新の参加者に基づいて決定します。
    次のターンに進む参加者の名前だけを述べます。
    参加者は連続して1ターン以上を取ってはいけません。
    
    次の参加者からのみ選択してください。
    - {RECEPTIONIST_NAME}
    - {ADDRESSUPDATER_NAME}
    - {OTHERTASKOPERATOR_NAME}
    
    次の参加者を選択するときは、常に次のルールに従ってください:
    - 初めは {RECEPTIONIST_NAME} から始めます。
    - ユーザーのリクエストに応じて、次の参加者を{RECEPTIONIST_NAME}以外から選択してください。
      ‐ 住所変更は {ADDRESSUPDATER_NAME} に振り分けます。
      - それ以外のリクエストは {OTHERTASKOPERATOR_NAME} に振り分けます。
    - 各担当者のタスクが完了したら一旦 {RECEPTIONIST_NAME} に対応を戻します。

    History:
    {{{{$history}}}}
    """,
)

In [ ]:
chat = AgentGroupChat(
    agents=[
        agent_receptionist,
        agent_addressupdater,
        agent_othertaskoperator,
    ],
    termination_strategy=KernelFunctionTerminationStrategy(
        agents=[
            agent_receptionist,
            agent_addressupdater,
            agent_othertaskoperator,
        ],
        function=termination_function,
        kernel=_create_kernel_with_chat_completion("termination"),
        result_parser=lambda result: any(keyword in str(result.value[0]).lower() for keyword in ["<return_to_user>", "<request_completed>"]),
        history_reducer=ChatHistoryTruncationReducer(target_count=1),
        history_variable_name="history",
        maximum_iterations=50,
    ),
    selection_strategy=KernelFunctionSelectionStrategy(
        initial_agent=agent_receptionist,
        function=selection_function,
        kernel=_create_kernel_with_chat_completion("selection"),
        result_parser=lambda result: str(result.value[0]) if result.value is not None else RECEPTIONIST_NAME,
        history_reducer=ChatHistoryTruncationReducer(target_count=1),
        history_variable_name="history",
    ),
)

In [ ]:
is_complete = False
while not is_complete:
    print()
    user_input = input("User > ").strip()
    if not user_input:
        continue

    if user_input.lower() == "exit":
        is_complete = True
        break

    if user_input.lower() == "reset":
        await chat.reset()
        print("[Conversation has been reset]")
        continue
    
    # Add the current user_input to the chat
    await chat.add_chat_message(message=user_input)

    try:
        async for response in chat.invoke():
            if response is None or not response.name:
                continue
            print(f"# {response.name.upper()}:\n{response.content}")
    except Exception as e:
        print(f"Error during chat invocation: {e}")

    # Reset the chat's complete flag for the new conversation round.
    chat.is_complete = False

In [ ]:
print(chat.history)

### OUTPUT SAMPLE
<span style="font-size: 85%">

**RECEPTIONIST:**<br>
こんにちは！ご来店ありがとうございます。本日はいかがいたしましょうか？ご希望の手続きやご相談をお伺いします。

**RECEPTIONIST:**<br>
本日は「住所変更」のお手続きのみ承っております。  
もしご住所の変更をご希望の場合は、対応させていただきます。他のお手続きについては、現在受付けておりませんのでご了承ください。

ご希望がございましたらお知らせください。

**ADDRESSUPDATER:**<br>
承知いたしました。  
住所変更のお手続きには、ご本人確認のため下記いずれかの書類のご提示が必要です。

- 運転免許証  
- 健康保険証  

お手元にご用意いただき、書類の画像または内容を提示してください。  
ご確認後、住所変更の手続きを進めさせていただきます。

**RECEPTIONIST:**<br>
ご指摘ありがとうございます。  
住所変更手続きを始める前に、まずご本人確認が必要です。

恐れ入りますが、銀行カード番号を教えていただけますか？  
番号を確認の上、登録情報（氏名・住所・電話番号）をお伝えしますので、内容が正しいかご確認ください。

**RECEPTIONIST:**<br>
ありがとうございます。

登録されているお客様情報をお伝えします。
ご氏名：マイクロ太郎
ご住所：港区港南 2-16-3
電話番号：03-4332-5300

この情報に相違がないかご確認ください。間違いがなければ、次に進みます。
よろしいでしょうか？

**RECEPTIONIST:**<br>
ご確認ありがとうございます。

それでは、住所変更手続きを進めさせていただきます。お手元にご本人確認書類（運転免許証または健康保険証）はございますか？  
もしご用意いただけましたら、書類の内容または画像を提示してください。  
準備ができましたらお知らせください。

**RECEPTIONIST:**<br>
ご本人確認のため、運転免許証番号を承りました。ありがとうございます。

それでは新しいご住所をお知らせいただけますか？  
新しいご住所を確認後、住所変更のお手続きを進めさせていただきます。

**RECEPTIONIST:**<br>
新しいご住所「港区港南 2-16-4」を承りました。

これより住所変更手続きを担当部署にて進めますので、少々お待ちください。  
手続きが完了しましたら、改めてご連絡いたします。


**RECEPTIONIST:**<br>
お待たせいたしました。

住所変更のお手続きは無事に完了しました。  
新しいご住所は「港区港南 2-16-4」となっております。

他にご不明な点やご要望はございますか？
</span>